In [ ]:
#  心超报告，调整正则后重新运行

create table if not exists af_ict.exclu_xin_ultrasound_1207 as 
select checks.ultrasound_report.visit_id,
       checks.ultrasound_report.visit_type,
       'ultrasound' as exclusion_source
from checks.ultrasound_report
where checks.ultrasound_report.check_name like '%%心%%'
  and checks.ultrasound_report.visit_id <> 0
  and (checks.ultrasound_report.check_conclusion like '%%瓣%%'
      or checks.ultrasound_report.check_conclusion like '%%风湿性%%'
      or checks.ultrasound_report.check_conclusion like '%%二尖%%'
      or checks.ultrasound_report.check_conclusion like '%%狭窄%%'
      or checks.ultrasound_report.check_conclusion like '%%中度二尖%%'
      or checks.ultrasound_report.check_conclusion like '%%重度二尖%%'
      or checks.ultrasound_report.check_conclusion like '%%风心%%');

# 满足超声检查类型(B)

create table if not exists af_ict.exclu_shi_ultrasound_1207 as 
select checks.ultrasound_report.visit_id,
       checks.ultrasound_report.visit_type,
       'ultrasound' as exclusion_source
from checks.ultrasound_report
where checks.ultrasound_report.visit_id <> 0
and (checks.ultrasound_report.check_name like '%%食%%'
       or checks.ultrasound_report.check_name like '%%食道%%'
       or checks.ultrasound_report.check_name like '%%TEE%%');

In [ ]:
sql_large_table = '''
                    select t_xinnei.*,
                            (case when t_fangchan.source is null then 0 else 1 end) as fangchan,
                            (case when t_fangchanshi.source is null then 0 else 1 end) as fangchanshi,
                            (case when t_ecg.source is null then 0 else 1 end) as ecg,
                            (case when t_banmobing.exclusion_source is null then 0 else 1 end) as banmobing,
                            (case when t_manzuchaosheng.exclusion_source is null then 0 else 1 end) as manzuchaosheng
        
                      from af_ict.distinct_unioned_visits t_xinnei
                             left join (select * from af_ict.af_criteria_diag_regexp
                                        union
                                        select * from af_ict.af_criteria_diag_synyi_code) t_fangchan
                               on t_xinnei.visit_id = t_fangchan.visit_id

                             left join af_ict.af_criteria_hospital_record t_fangchanshi
                             on t_xinnei.visit_id = t_fangchanshi.visit_id

                             left join af_ict.af_criteria_nlp_ecg_report t_ecg
                             on t_xinnei.visit_id = t_ecg.visit_id

                             left join af_ict.exclu_xin_ultrasound_1207 t_banmobing
                             on t_xinnei.visit_id = t_banmobing.visit_id

                             left join af_ict.exclu_shi_ultrasound_1207 t_manzuchaosheng
                             on t_xinnei.visit_id = t_manzuchaosheng.visit_id

                      where t_xinnei.visit_id is not null;                     
'''